# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902002


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.

In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<01:03,  2.19s/it]

Rendering models:  10%|▉         | 3/31 [00:09<01:27,  3.12s/it]

Rendering models:  13%|█▎        | 4/31 [00:10<01:05,  2.42s/it]

Rendering models:  16%|█▌        | 5/31 [00:11<00:53,  2.04s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:27<00:02,  1.01it/s]

Rendering models:  94%|█████████▎| 29/31 [00:29<00:02,  1.09s/it]

Rendering models:  97%|█████████▋| 30/31 [00:29<00:00,  1.04it/s]

Rendering models: 100%|██████████| 31/31 [00:30<00:00,  1.09it/s]

equidad1                              0.000437
Sammy_Lawrence11                      0.091442
not-logged-in-6f58cd197f9447fe73f1    0.002031
YugnatD                               0.000723
not-logged-in-bc8d75575cbd7912514f    0.001574
not-logged-in-204fc099209d1816fe0b    0.000570
not-logged-in-a87793e85210b5433279    0.000357
simonemarcorini                       0.000798
not-logged-in-7fb5684ea2b60935e324    0.000571
jumpy26                               0.000942
kayleebug2017                         0.001094
not-logged-in-7e1155a225937ff5fedb    0.093141
not-logged-in-a92559927bfd05e023e7    0.000869
Marra                                 0.001213
not-logged-in-1d7e72cf6d080c497c05    0.012575
not-logged-in-9a45911eb71184fe7c7f    0.000636
Planetme                              0.001131
pangeli5                              0.000706
not-logged-in-04ed9ae880f18762cd9e    0.022468
not-logged-in-1cf1343051a1dd8ef2d0    0.005630
Lavadude                              0.029616
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())